In [52]:
import pandas as pd
import requests
from urllib.parse import urlencode

In [53]:
# Чтение данных из облака
def download_link(public_key):

    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    return download_url

In [54]:
df = pd.read_csv(download_link('https://disk.yandex.ru/d/GOLy9SzTS7OQnQ'),
    parse_dates=['InvoiceDate'])

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [55]:
df.dtypes

InvoiceNo               object
CustomerCode            object
InvoiceDate     datetime64[ns]
Amount                 float64
dtype: object

In [56]:
df['InvoiceNo'] = df['InvoiceNo'].apply(str)
df['CustomerCode'] = df['CustomerCode'].apply(str)

Какое максимальное кол-во покупок было совершено одним пользователем?

In [57]:
df.groupby('CustomerCode', as_index=False)\
    .agg({'InvoiceNo': lambda x: len(x)})\
    .sort_values('InvoiceNo', ascending=False)\
    .head(1)

,CustomerCode,InvoiceNo
89388,19057820,204


In [58]:
# Max date in the DF
max_date = df['InvoiceDate'].max()

In [59]:
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [60]:
rfm_df = df.groupby('CustomerCode', as_index=False)\
    .agg(recency=('InvoiceDate', lambda x: (max_date - x.max()).days),
         frequency=('Amount', lambda x: len(x)),
         monetary=('Amount', 'sum'))

In [61]:
df['CustomerCode'].nunique()

123733

In [62]:
rfm_df.head()

,CustomerCode,recency,frequency,monetary
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88


Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М?
Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F? <br>
Какое максимальное количество дней может пройти с момента последней покупки для того, чтобы пользователь попал в класс 2 в подсегменте R?

In [63]:
quantiles_table = rfm_df.quantile(q=[0.25, 0.5, 0.75])

In [64]:
quantiles_table

,recency,frequency,monetary
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [65]:
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4

def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1


In [66]:
rfm_df['R_Quartile'] = rfm_df['recency'].apply(RClass, args=('recency',quantiles_table))

rfm_df['F_Quartile'] = rfm_df['frequency'].apply(FMClass, args=('frequency',quantiles_table))

rfm_df['M_Quartile'] = rfm_df['monetary'].apply(FMClass, args=('monetary',quantiles_table))

rfm_df['RFMClass'] = rfm_df.R_Quartile.map(str) + rfm_df.F_Quartile.map(str) + rfm_df.M_Quartile.map(str)

In [67]:
rfm_df.head()

,CustomerCode,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFMClass
0,02213019,19,1,1609.20,4,4,3,443
1,02213042,22,3,9685.48,4,2,1,421
2,02213071,29,1,415.00,4,4,4,444
3,02213088,23,1,305.00,4,4,4,444
4,02213092,25,1,1412.88,4,4,3,443


In [68]:
rfm_df.groupby('RFMClass', as_index=False)\
    .nunique()\
    .query("RFMClass == '311'")

,RFMClass,CustomerCode,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile
32,311,1609,8,16,1607,1,1,1


В каком RFM-сегменте самое большое кол-во пользователей? <br>
В каком RFM-сегменте самое маленькое кол-во пользователей? <br>
Какое количество пользователей попало в самый малочисленный сегмент?

In [73]:
rfm_df.head()

,CustomerCode,recency,frequency,monetary,R_Quartile,F_Quartile,M_Quartile,RFMClass
0,02213019,19,1,1609.20,4,4,3,443
1,02213042,22,3,9685.48,4,2,1,421
2,02213071,29,1,415.00,4,4,4,444
3,02213088,23,1,305.00,4,4,4,444
4,02213092,25,1,1412.88,4,4,3,443


In [70]:
grouped_rfm = rfm_df.groupby('RFMClass', as_index=False)\
    .agg(nunique_users=('CustomerCode', 'nunique'))\
    .sort_values('nunique_users', ascending=False)

In [71]:
grouped_rfm.loc[grouped_rfm.nunique_users.idxmax()]

RFMClass           444
nunique_users    10624
Name: 63, dtype: object

In [72]:
grouped_rfm.loc[grouped_rfm.nunique_users.idxmin()]

RFMClass         414
nunique_users      2
Name: 51, dtype: object